# Preprocessing: Cropping

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import os
import tifffile as tiff
from tifffile import imsave
from tifffile import TiffWriter
from PIL import Image
import natsort
from ipywidgets import interact, interactive, fixed, interact_manual
import time

### First, read an example image file

In [2]:
# Find the data

data_folder = '/users/charles.fieseler/shared_projects/ForCharlie/27082020_freelyimmobilized_trial2/'
data_file = 'ZIM2051_trial_21_w2dual2-GFP-mCherry-SD-DV--488Ex--561Ex_t9.stk'
# data_file = 'ZIM2051_trial_11_w2dual2-GFP-mCherry-SD-DV--488Ex--561Ex_t9.stk'

full_fname = os.path.join(data_folder, data_file)


In [3]:
# Actually read the data in
im = tiff.imread(full_fname)

In [4]:
def myplt(i):
    plt.figure(figsize=[15,15])
    plt.imshow(im[i,...])
    plt.title(f"Slice {i}")
    plt.colorbar()
interact(myplt, i=(0, 33))

interactive(children=(IntSlider(value=16, description='i', max=33), Output()), _dom_classes=('widget-interact'…

<function __main__.myplt(i)>

## Determine the crop via GUI

Uses this package:
https://openbits.app/posts/python-interactive-cropping/

pip install interactivecrop

In [5]:
from interactivecrop.interactivecrop import main as crop

In [6]:
# Use same fname as the example

data_folder = '/users/charles.fieseler/shared_projects/ForCharlie/27082020_freelyimmobilized_trial2/'
data_file = 'ZIM2051_trial_21_w2dual2-GFP-mCherry-SD-DV--488Ex--561Ex_t9.stk'
# data_file = 'ZIM2051_trial_11_w2dual2-GFP-mCherry-SD-DV--488Ex--561Ex_t9.stk'

full_fname = os.path.join(data_folder, data_file)

# Actually read the data in
im = tiff.imread(full_fname)

In [8]:
# Display only one slice
which_slice = 15

w = crop([im[which_slice,...]])
display(w)

GridBox(children=(Dropdown(description='Img Name:', layout=Layout(grid_area='im_selector', width='100%'), opti…

## ACTUALLY SAVE THE CROP SIZE

In [90]:
crop_sz = [w.children[1].value, w.children[2].value]
print(crop_sz)

[(286, 590), (373, 516)]


## Using the crop sizes, save as ome.tiff

In [91]:
data_parent_folder = '/users/charles.fieseler/shared_projects/ForCharlie/'
data_folder = '27082020_freelyimmobilized_trial2'

# Input
input_folder = os.path.join(data_parent_folder, data_folder)

# New, output folder
o_name = 'ome_tiff'
output_folder = os.path.normpath(os.path.join(data_parent_folder, data_folder, '..', f'{data_folder}_{o_name}_{int(time.time())}'))

##select z slices to save (only works on axis=0)
# fri=0
# frf=33

In [98]:


if not os.path.exists(output_folder):
    os.makedirs(output_folder)
else:
    print(f"Folder {output_folder} already exists!")
filename=os.path.splitext(os.listdir(input_folder)[0])[0]
print('Reading all files in directory {}'.format(filename))
# IM=tiff.imread(os.path.join(i_directory,filename))
#fold increase for better visuTiffWriteron of the very 'short' z axis. Set to 1 for no increase.
#set it to have the same pixel size as XY (0.3um vs 1um zstep size)
# fold_increase=3


# #creates an empty matrix for the lower right part of the image. It should have the dimensions of Z-stacks times the fold increase. (now 150)
# corner_matrix=np.full((fold_increase*IM.shape[0],fold_increase*IM.shape[0]),100, dtype='uint16')

Folder /users/charles.fieseler/shared_projects/ForCharlie/27082020_freelyimmobilized_trial2_ome_tiff_1601906121 already exists!
Reading all files in directory ZIM2051_trial_21


In [99]:

# Setup
do_crop = True
if do_crop:
    print("Saving files with crop: ", crop_sz)
else:
    print("Saving files with no crop")
crop_width, crop_height = crop_sz

sorted_fnames = natsort.natsorted(os.listdir(input_folder))

Saving files with crop:  [(286, 590), (373, 516)]


In [103]:
# Finally, actually import
# Note: name the ome_tif using the metadata file from the .stk folder, i.e. the .nd file

# Note: This saves via concatenation

with TiffWriter(os.path.join(output_folder,filename+'.tif'), bigtiff=True) as tif_w:
    for i, this_file in enumerate(sorted_fnames):
        if i % 10 == 0:
            print(f'Reading file {i}/{len(sorted_fnames)}')
#         if i > 13:
#             break
        if this_file.endswith(".stk"):
            this_full_fname = os.path.join(input_folder, this_file)
            IM = tiff.imread(this_full_fname)
            # TODO: tiff.ome to get metadata
#             ome_xml_Metadata = tiff.TiffFile(this_full_fname).ome_metadata
#             print(ome_xml_Metadata)
            
            # Note: shape is zhw
            if do_crop:
                final_image = IM[:,crop_height[0]:crop_height[1], crop_width[0]:crop_width[1]]
            else:
                final_image = IM

            # convert to 16bit
            #final_image16bit = np.int16(final_image)

            # data = numpy.random.randint(0, 255, (5, 2, 3, 301, 219), 'uint8')
            
            # TODO: implement a basic buffer
            tif_w.save(final_image, compress=0, photometric='minisblack')
#             tif_w.save(final_image, compress=0, photometric='minisblack', contiguous=True)
#             clear_output()
print('end')

Reading file 0/9091
Reading file 10/9091
Reading file 20/9091
Reading file 30/9091
Reading file 40/9091
Reading file 50/9091
Reading file 60/9091
Reading file 70/9091
Reading file 80/9091
Reading file 90/9091
Reading file 100/9091
Reading file 110/9091
Reading file 120/9091
Reading file 130/9091
Reading file 140/9091
Reading file 150/9091
Reading file 160/9091
Reading file 170/9091
Reading file 180/9091
Reading file 190/9091
Reading file 200/9091
Reading file 210/9091
Reading file 220/9091
Reading file 230/9091
Reading file 240/9091
Reading file 250/9091
Reading file 260/9091
Reading file 270/9091
Reading file 280/9091
Reading file 290/9091
Reading file 300/9091
Reading file 310/9091
Reading file 320/9091
Reading file 330/9091
Reading file 340/9091
Reading file 350/9091
Reading file 360/9091
Reading file 370/9091
Reading file 380/9091
Reading file 390/9091
Reading file 400/9091
Reading file 410/9091
Reading file 420/9091
Reading file 430/9091
Reading file 440/9091
Reading file 450/9091

In [86]:
im.shape

(33, 655, 885)

# Reading the ome.tiff file back in

In [1]:
import tifffile
import os
from DLC_for_WBFM.utils.video_and_data_conversion.xml_utils import *

In [44]:
#pathtofile
data_parent_folder = '/users/charles.fieseler/shared_projects/ForCharlie/'
data_folder = '27082020_freelyimmobilized_trial2_ome_tiff_1601906121'
data_file = os.path.join(data_parent_folder, data_folder, 'ZIM2051_trial_21.tif')

# Input
tmp = []
with tifffile.TiffFile(data_file, multifile=False) as tif:
    tmp = tif.pages[0].shape
    print(tif.series[0].shape)
#     print(tif.pages[0].__dict__)
#     im = tif.asarray()
#     omexmlMetadataString = tif.ome_metadata
print(tmp)

(9090, 33, 143, 304)
(143, 304)


In [41]:
first_frame = tifffile.imread(data_file, key=0)
first_frame.shape

(143, 304)

In [2]:
# Tests for adding metadata back in

# First, read one of the original .stk files
data_parent_folder = '/users/charles.fieseler/shared_projects/ForCharlie/'
data_folder = '27082020_freelyimmobilized_trial2'
# data_file = os.path.join(data_parent_folder, data_folder, 'ZIM2051_trial_21.nd')
data_file = os.path.join(data_parent_folder, data_folder, 'ZIM2051_trial_21_w2dual2-GFP-mCherry-SD-DV--488Ex--561Ex_t2365.stk')

stk_dat = tifffile.TiffFile(data_file, multifile=False)

In [8]:
print(stk_dat.asarray().shape)
stk_dat.stk_metadata
print(stk_dat.)

(33, 655, 885)
None


# Save a video (single plane)

In [1]:
from DLC_for_WBFM.utils.video_and_data_conversion.video_conversion_utils import *
import tifffile
import os

In [4]:
# Locations of input and output
data_parent_folder = '/users/charles.fieseler/shared_projects/ForCharlie/'
data_folder = '27082020_freelyimmobilized_trial2_ome_tiff_1601906121'
input_fname = os.path.join(data_parent_folder, data_folder, 'ZIM2051_trial_21.tif')

target_folder = '/users/charles.fieseler/dat/'
output_fname = os.path.join(target_folder, 'ZIM2051_trial_21.ome.tiff')

In [5]:
# Settings
which_slice = 15
num_frames = 1000

In [6]:
# Creates a smaller ome.tiff file with only these frames and slice
dat = write_video_from_ome_file(num_frames, input_fname, output_fname, which_slice=which_slice, out_dtype='uint32')

AttributeError: 'NoneType' object has no attribute 'keys'

In [3]:
# Creates the DLC-compatible video
write_numpy_as_avi(dat, fname=out_fname) # from above OME-TIFF

Help on function write_video_from_ome_file in module DLC_for_WBFM.utils.video_and_data_conversion.video_conversion_utils:

write_video_from_ome_file(num_frames, video_fname, out_fname, out_dtype='uint16', which_slice=None)
    Takes a video filename, which is a large ome-tiff file, and saves a smaller file in the folder given by 'out_folder'

